In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, GaussianDropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU 
from tensorflow.python.keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

In [38]:
class GAN():
    def __init__(self):
        self.N = 60000
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.dropout = 0.5
        lengthscale = 1e-2
        self.reg = lengthscale**2 * (1 - self.dropout) / (2. * self.N )
        
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.summary()
        
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        self.generator.summary()

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        print(img.shape)

        self.discriminator.trainable = False
        validity = self.discriminator(img)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        noise = Input(shape=(self.latent_dim,))
        model = Dense(256 * 7 * 7, input_dim=self.latent_dim, kernel_regularizer=l2(self.reg))(noise)
        model = Dropout(0.5)(model,training=True)
        model = Activation('relu')(model)
        model = Reshape((7, 7, 256))(model)
        model = Conv2DTranspose(128, kernel_size=(5, 5), strides=(1, 1), padding='same', kernel_regularizer=l2(self.reg))(model)
        model = Dropout(0.5)(model,training=True)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Conv2DTranspose(64, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_regularizer=l2(self.reg))(model)
        model = GaussianDropout(0.5)(model,training=True)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Conv2DTranspose(1, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_regularizer=l2(self.reg))(model)
        #model = Dropout(0.5)(model,training=True)
        #model = BatchNormalization(momentum=0.8)(model)
        model = Activation('tanh')(model)
        return Model(noise, model)
    
    def build_discriminator(self):
        img = Input(shape=self.img_shape)
        model = Conv2D(64, kernel_size=5, strides=2, input_shape=self.img_shape, padding="same", kernel_regularizer=l2(self.reg))(img)
        model = Dropout(0.5)(model,training=True)
        model = LeakyReLU(alpha=0.2)(model)
        model = Conv2D(128, kernel_size=5, strides=2, padding="same", kernel_regularizer=l2(self.reg))(model)
        model = Dropout(0.5)(model,training=True)
        model = LeakyReLU(alpha=0.2)(model)
        model = Flatten()(model)
        output = Dense(1, activation='sigmoid')(model)     
        return Model(img, output)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        (X_train, _), (_, _) = mnist.load_data()
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            if epoch % sample_interval == 0:
               self.sample_images(epoch)
            
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
    
    def predict(self, no_samples):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        for k in range(no_samples):
            
            gen_imgs = self.generator.predict(noise)
            gen_imgs = 0.5 * gen_imgs + 0.5
            fig, axs = plt.subplots(r, c)
            cnt = 0
            for i in range(r):
                for j in range(c):
                    axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                    axs[i,j].axis('off')
                    cnt += 1
            fig.savefig("images/%d.png" % k)
            plt.close()
            
    def average(self, no_samples):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        for k in range(no_samples):            
            generated_imgs = self.generator.predict(noise)
            generated_imgs = 0.5 * generated_imgs + 0.5
            gen_imgs += generated_imgs
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%dfinal.png" % k)
        plt.close()

In [39]:
gan = GAN()
gan.train(epochs=28000, batch_size=128, sample_interval=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 14, 14, 64)        1664      
_________________________________________________________________
dropout_60 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 7, 7, 128)         204928    
_________________________________________________________________
dropout_61 (Dropout)         (None, 7, 7, 128)         0         
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 7, 7, 128)         0         
__________

KeyboardInterrupt: 

In [31]:
gan.predict(20)
gan.average(1000)
gan.average(10)
gan.average(100)

In [30]:
self.combined.summary()

NameError: name 'self' is not defined